In [2]:
import pathlib
from Utils.wetland_selection    import get_top_scenes
from Utils.treatment_regression import logistic_process
from Utils.outcome_syndata_generator import generate_synthetic_outcome
from Utils.treatment_syndata_generator import generate_synthetic_treatment
from Utils.outcome_regression import regression_process
from Utils.convolution import make_exp_kernel
from Utils.outcome_post import generate_post
import numpy as np

In [3]:
# top scenes by wetland development
top_df = get_top_scenes(num_scenes=3566)
scene_ids = top_df['scene'].astype(str).tolist()

# define folder paths for rasters
DATA_DIR = pathlib.Path('/Users/sayedmorteza/Library/CloudStorage/Box-Box/Hetwet_Data')         

# folders
folders = {
    'wet': DATA_DIR / 'WETLAND_DEV_1996_2016',
    'dem': DATA_DIR / 'DEM',
    'cap': DATA_DIR / 'CAPITAL_1996',
    'claims_96': DATA_DIR / 'LOG_CLAIMS_1996',
    'claims_16': DATA_DIR / 'LOG_CLAIMS_2016',

}

In [3]:
# estimate treatment logistic regression parameters
theta = 0.5  # classification threshold
logit_res = logistic_process(
    scene_ids=scene_ids,
    folders=folders,
    threshold=theta,
    regularization_C=1.0,
    noise_type='gaussian',
    noise_sd=0.005,
    verbose=False
)

In [4]:
# generate synthetic treatment
generate_synthetic_treatment(
    scene_ids=scene_ids,
    folders=folders,
    logit_pipe=logit_res,
    threshold=theta,
    noise_sd=0.005,
    noise_type='gaussian'
)


In [3]:
# estimate outcome linear regression parameters
outcome_reg = regression_process(
    scene_ids=scene_ids,
    folders=folders,
    noise_sd=0.1,
    noise_type='gaussian',
    verbose=False
)

In [4]:
# generate synthetic baseline outcomes:
generate_synthetic_outcome(
    scene_ids = scene_ids,
    folders = folders,
    reg_pipe = outcome_reg,
    target_shape = (8, 8),
    noise_type = "gaussian",
    noise_sd = 0.1,
    results_dir = None
)

In [4]:
# build your spillover kernel
CELL_SIZE = 30 # cell size in meters
LAMBDA    = 100.0
TRUNCATE = 1
KERNEL    = make_exp_kernel(lam_m=LAMBDA, cell_size_m=CELL_SIZE, truncate=TRUNCATE)
print(f"Number of Affected Nearby Squares: {KERNEL.shape}")

Number of Affected Nearby Squares: (7, 7)


In [5]:
generate_post(
    scene_ids=scene_ids,
    folders=folders,
    KERNEL=KERNEL,
    Bbase = 0.5,
    Beta1 = 0.05,
    Beta2 = 2.5
)

/opt/miniconda3/envs/imageTRT/lib/python3.10/site-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/opt/miniconda3/envs/imageTRT/lib/python3.10/site-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/opt/miniconda3/envs/imageTRT/lib/python3.10/site-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/opt/miniconda3/envs/imageTRT/lib/python3.10/site-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset 